# UNIVARIATE ANALYSIS

**Part 1. Filters** 

Feature analysis for regression models takes a lot of time and manual work, but not all the features are worth to do so.
In this step, we can exclude up to 90% of the features, depends on the data.
1. `find_informative_features`: Designed for sparse data, the function tells whether **being null or not** is a good indicator.
2. `anova_analysis`: Rank features by the value of f-test and chi-squared test. Then keep features that distributes more differently in two groups (target=1 and target=0).

**Part 2. Thresholds** (for numerical features)

When the original distribution is not ideal for linear/non-linear transformation, we group data into bins that optimize the linear trend of event rates. The self-defined class we used here is called `Binner`.

---

### Import tools, Environment Settings

In [ ]:
import numpy as np                                
import pandas as pd
import copy

# custom libraries
from binner import NumericFeatureBinner, NominalFeatureBinner

# Data preparation

### Import

In [ ]:
data = read_csv('....csv')
print("data shape: ", data.shape)

### Preprocess

In [ ]:
def preprocess(data):

    df = copy.deepcopy(data)
    
    # set target value name = 'y'
    # set target value to binary 
    df['y'] = df['label']
    df = df.replace({'y':{'a':0, 'b':1, 'c':1, 'd':1}})
    
    # drop unwanted columns
    drop_key = ... # list of columns
    df = df.drop(labels=drop_key, axis=1)

    print('============== processed data info =============')
    print(df.shape)
                
    return df   

In [ ]:
data = data.preprocess(data)

# Univariate analysis

### I. Informative

The function finds out the features that are more informative in a sparse dataset. In each feature, assume null value = 0 and 1 otherwise. The porportion of events in which feature = 1 tells whether being null or not is a good indicator of event.

For example:
* Feature A has 80% null values, and let target = 0 or 1. 
* The function will take the rest 20% records, sum the target and divide by the count of datapoints. It's also called "target rate".
* The funtion returns features that pass the given threshold.

In [ ]:
def find_informative_features(data, target_rate_threshold):
    stats = {}

    # select features to be passed to this filter
    f_list = data.columns.tolist()
    f_list = ... # customize
    
    for f in f_list:
        y = data['y']
        notna_data = data[data.notna()]
        notna_y = y[data.notna()]
        
        stats[f] = {
            'feature': f,
            'notna_count': notna_data.count(),
            'notna_target': notna_y.sum(),
            'notna_target_rate': round(notna_y.sum()/notna_data.count(),4)
        }
    
    # turn stats into table
    table = pd.DataFrame.from_dict(stats)
    table = table.T.reset_index(drop=True)
    
    # keep those strong features
    filtered = table[table.notna_target_rate >= target_rate_threshold].sort_values(by=['notna_target_rate','notna_target'], ascending=False)
    filtered = pd.concat([filtered, keep])
    filtered = filtered.reset_index(drop=True)
    
    return filtered


In [ ]:
informative_table = find_informative_features(data, 0.05)
informative_features = table["feature"].tolist()

### II. ANOVA test

The idea of ANOVA test is to examine whether the population of multiple independent samples are significantly different. 

Here, we seperate features into two sample group according to their `Y` (Y is binary). 
If the ANOVA test finds that the population of two sample groups are different, then we expect this feature to be a good indicator to seperate `Y=0` and `Y=1`.

For different data types, we apply different test to measure the variance between two smaple's population.
- Continuous: `f_classif`
- Categorical, Nominal: `chi2` 

---

(Explanation of ANOVA concepts, Chinese version)

**【前提假設】**

ANOVA test(變異數分析)的主要功能是用來**檢定多組相互獨立樣本的母體平均數是否具有顯著差異**(可以理解為檢定多組樣本間是否不同)；例如：亞洲各國家的成人平均身高是否相同。
在進行檢驗前，我們希望確定每組獨立樣本的平均數的確能夠被拿來互相比較；因此各組樣本的母體除了需要符合常態分布外，還希望其資料離散分布的狀況能具有相似性，也就是說，`樣本的母體變異數必須具有同質性`。
嚴格來說，進行變異數分析之前，我們會先進行樣本母體變異數的同質性檢定，若各樣本的母體變異數皆具有同質性，就可以使用變異數分析來進行之後的檢定。
但我們這邊因為只用作變數間的比較及排序，所以跳過此一步驟。

**【量化樣本間的差異：利用樣本變異程度】**
- 組間變異(Between-group variation)：不同組樣本之間的資料變異程度
- 組內變異(Within-group variation)：各組樣本本身資料間的變異程度

得到樣本的變異資訊後，就可以利用組間變異與組內變異的相對大小來檢定不同組樣本之間母體平均數的差異。
`組間差異越大`、`組內差異越小`的時候，表示這個變數在Y=0和Y=1的人身上的`數據長相差異越大(f-value越大)`。
另一種角度說就是這個變數的值可以很好的區別Y=0和Y=1的人。(->how well this feature discriminates between two classes)

**【針對變數類型採用不同方法】**
- 連續數值型變數：f_classif（F檢定）
- 離散且不連續、類別型變數：chi2（卡方檢定）


[參考資料: F-value](https://datascience.stackexchange.com/questions/74465/how-to-understand-anova-f-for-feature-selection-in-python-sklearn-selectkbest-w)
 / [ANOVA test 易踩誤區](https://towardsdatascience.com/mistakes-in-applying-univariate-feature-selection-methods-34c43ce8b93d)
 / [ANOVA 變異數分析基本概念](https://yourgene.pixnet.net/blog/post/118650399-%E8%AE%8A%E7%95%B0%E6%95%B8%E5%88%86%E6%9E%90anova)

In [ ]:
def anova_analysis(data, percentile, plot=True):
    from sklearn.feature_selection import SelectPercentile, f_classif, chi2
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import Normalizer

    def conduct_anova(data, ftype, plot, percentile):
        
        # data type inspector
        if ftype == 'numerical':
            f_list = list(set(data.columns[data.dtypes != 'object']) & set(data.columns[data.dtypes != 'bool']))
            f_list.remove('y')
        elif ftype == 'categorical':
            f_list = list(set(data.columns[data.dtypes == 'object']) | set(data.columns[data.dtypes == 'bool']))
            f_list.remove('y')
        
        X = data[f_list]
        X = X.fillna(0)
        y = data['y'] 

        # normalize
        if ftype == 'numerical': 
            normalizer = Normalizer()
            X = normalizer.fit_transform(X)
            X = pd.DataFrame(X, columns = f_list)
        
        # turn str and bool to numerical values (for chi-test)
        X = X.apply(lambda x: pd.factorize(x)[0] if x.dtype == 'bool' or x.dtype == 'object' else x)
        f_cnts = X.shape[-1]
        if f_cnts == 0:
            return None    

        if ftype == 'numerical':
            selector = SelectPercentile(f_classif, percentile)
        elif ftype == 'categorical':
            selector = SelectPercentile(chi2, percentile)
            
        selector.fit(X, y)
        keep_indices = selector.get_support(indices=True)
        keep_features, score, p_value = np.array(X.columns[keep_indices]), selector.scores_[keep_indices], selector.pvalues_[keep_indices]
        stats = pd.DataFrame(data = [keep_features, score, p_value]).T
        stats.columns= ["feature", "score (F or chi)", "p_value"]
        stats['dtype'] = [ftype] * stats.shape[0]
        
        if plot:
            bar1 = score
            bar2 = p_value
            indx = np.arange(len(bar1))  

            fig, (ax2, ax3) = plt.subplots(nrows=2, ncols=1, constrained_layout=True)
            ax2.plot(indx, bar1)
            ax3.plot(indx, bar2)
            fig.suptitle('**************************  {}, {}  **************************'.format(d, ftype), fontsize=16)
            ax2.set_title("score(F or chi)")
            ax3.set_title("p-value")
        
        return stats
    
    
    result = pd.DataFrame()
    for ftype in ['numerical', 'categorical']:
        stats = conduct_anova(data, ftype, plot, percentile)
        if stats is not None:
            result = pd.concat([result, stats], ignore_index=True)
            print(d, ftype, 'picked ---',stats.shape[0])
    
    return result


In [ ]:
anova_table = anova_analysis(data, percentile=50, plot=False)
anova_features = anova_table["feature"]

# Binner: get_threshold

In [ ]:
# concatenate all the features that passed filter I and II; these are features that are recommended to go on further analysis
selected_features = list(set(anova_features) | set(informative_features))
print("number of features after filter:", len(selected_features))

Apply `Binner`, a self-defined class to find the optimized threshold.


In [ ]:
def get_threshold(data, selected_features, plot_binner=False):
    from binner import NumericFeatureBinner
    from binner import NominalFeatureBinner
    
    result = pd.DataFrame()
    stats = {}
    f_list = [f for f in selected_features if f in data.columns.tolist()]
    numeric_f_list = list(set(data.columns[data.dtypes != 'object']) & set(data.columns[data.dtypes != 'bool']))
    category_f_list = list(set(data.columns[data.dtypes == 'object']) | set(data.columns[data.dtypes == 'bool']))

    for f in f_list:
        X = pd.DataFrame(data[f]).reset_index(drop=True)
        y = pd.DataFrame(data['y']).reset_index(drop=True)
                
        # Categorical features
        if f in category_f_list:
            binner = NominalFeatureBinner()
            binner.fit(X, y, method='order', criteria='event_rate(%)')
            
            bin_table = binner.get_performance_df({f: [X, y]})
            if 'NaN' in bin_table.columns.tolist():
                bin_table = bin_table.drop(columns=['NaN'])
                
            bin_table = bin_table.T
            bin_table['covered_bad_rate'] = bin_table['event_rate(%)'].astype(float)/100
            max_bin = bin_table['covered_bad_rate'].idxmax()
            
            stats[f] = {
                        'feature': f,
                        'covered_count': bin_table.loc[max_bin, 'count'],
                        'covered_bad': bin_table.loc[max_bin, 'event'],
                        'covered_bad_rate': bin_table.loc[max_bin, 'covered_bad_rate'],
                        'condition':bin_table.loc[max_bin, 'fields']
                    }
            if plot_binner:
                binner.plot({f: [X, y]})
        
        # Numerical features
        if f in numeric_f_list:
            binner = NumericFeatureBinner()
            binner.fit_auto_merge_bins(X, y, max_bins=20, method='tree', criteria='event_rate(%)')

            bin_table = binner.get_performance_df({f: [X, y]})
            if 'NaN' in bin_table.columns.tolist():
                bin_table = bin_table.drop(columns=['NaN'])
                
            # filter again
            # if (event rate in max bin/event rate in averge) is less than 1.5 => drop the feature
            max_box = bin_table.loc['event_rate(%)'].astype(float).max()/100
            min_box = bin_table.loc['event_rate(%)'].astype(float).min()/100
            mean_box = bin_table.loc['event'].sum()/bin_table.loc['count'].sum()
            bin_table = bin_table.T
            bin_table['covered_bad_rate'] = bin_table['event_rate(%)'].astype(float)/100
            
            if (max_box/mean_box) >= 1.5:
                max_bin = bin_table['covered_bad_rate'].idxmax()
                condition = bin_table.loc[max_bin, 'upper_bound']
                if condition == 'else':
                    condition = '>=' + str(bin_table.loc[max_bin, 'min'])
                else:
                    condition = '<=' + str(bin_table.loc[max_bin, 'upper_bound'])
                
                name = f + '_bad'
                stats[f] = {
                            'feature': name,
                            'covered_count': bin_table.loc[max_bin, 'count'],
                            'covered_bad': bin_table.loc[max_bin, 'event'],
                            'covered_bad_rate': bin_table.loc[max_bin, 'covered_bad_rate'],
                            'condition': condition
                        }
                if plot_binner:
                    binner.plot({f: [X, y]})
                                
            # if (event rate in min bin/event rate in average) is larger than 0.5 => drop the feature
            if ((min_box/mean_box) <= 0.5) | (min_box == 0):
                min_bin = bin_table['covered_bad_rate'].idxmin()
                condition = bin_table.loc[min_bin, 'upper_bound']
                if condition == 'else':
                    condition = '>=' + str(bin_table.loc[min_bin, 'min'])
                else:
                    condition = '<=' + str(bin_table.loc[min_bin, 'upper_bound'])
                
                name = f + '_good'
                stats[f] = {
                            'feature': name,
                            'covered_count': bin_table.loc[min_bin, 'count'],
                            'covered_bad': bin_table.loc[min_bin, 'event'],
                            'covered_bad_rate': bin_table.loc[min_bin, 'covered_bad_rate'],
                            'condition': condition
                        }

                if plot_binner:
                    binner.plot({f: [X, y]})

    # turn stats into table
    df = pd.DataFrame.from_dict(stats).T.reset_index(drop=True)
    result = pd.concat([result, df], ignore_index=True)
    result = result.sort_values(by=['covered_bad_rate','covered_bad'], ascending=False)
    result = result.reset_index(drop=True)
        
    return result

In [ ]:
result = get_threshold(data, selected_features, plot_binner=False)